In [1]:
import time
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from holidays.countries import US
import numpy as np

### DEFINE

In [2]:
# 讀取檔案位置

# 初始年月份
set_data_year = 2020 #start_year
set_data_month = 1 #start_month
end_year = 2020
end_month = 12

# 檔案位置
Site = './data/'

# 檔案輸出位置
custom_output_path_dfTime = './dfTime/test'
custom_output_path_dfTLC = './dfTLC/test'

# 年月
start_year = set_data_year
start_month = set_data_month

datasite_fhvhv = []
datasite_yellow = []
for year in range(start_year, end_year+1):  # 修改为你的年份范围
    if year == start_year:
        if year < end_year :
            for month in range(start_month, 13):  # 生成1到12的月份
                filename = f'{Site}fhvhv_tripdata_{year}-{month:02}.parquet'
                datasite_fhvhv.append(filename)
                filename = f'{Site}yellow_tripdata_{year}-{month:02}.parquet'
                datasite_yellow.append(filename)                
        elif year == end_year:
            for month in range(start_month, end_month+1):  # 生成1到12的月份
                filename = f'{Site}fhvhv_tripdata_{year}-{month:02}.parquet'
                datasite_fhvhv.append(filename)    
                filename = f'{Site}yellow_tripdata_{year}-{month:02}.parquet'
                datasite_yellow.append(filename)   
    elif year > start_year:
        if year < end_year :
            for month in range(1, 13):  # 生成1到12的月份
                filename = f'{Site}fhvhv_tripdata_{year}-{month:02}.parquet'
                datasite_fhvhv.append(filename)
                filename = f'{Site}yellow_tripdata_{year}-{month:02}.parquet'
                datasite_yellow.append(filename)   
        elif year == end_year:
            for month in range(1, end_month+1):  # 生成1到12的月份
                filename = f'{Site}fhvhv_tripdata_{year}-{month:02}.parquet'
                datasite_fhvhv.append(filename)         
                filename = f'{Site}yellow_tripdata_{year}-{month:02}.parquet'
                datasite_yellow.append(filename)   



print("done時間:", datetime.datetime.now())

done時間: 2023-08-16 13:45:59.652343


### 留下需要特徵 & 移除不在當月資料
    HVFHV: 'hvfhs_license_num','pickup_datetime','dropoff_datetime','PULocationID','DOLocationID','trip_miles','trip_time','base_passenger_fare'
    
    Yellow: 'tpep_pickup_datetime','tpep_dropoff_datetime','trip_distance','PULocationID','DOLocationID','fare_amount'

In [3]:
# 記錄開始時間
start_time = time.time()


# 建立一個空的DataFrame來存放合併後的數據
df_fhvhv = pd.DataFrame()
df_yellow = pd.DataFrame()

data_year = set_data_year
data_month = set_data_month
temp0_total_fhvfv = 0
temp1_total_fhvfv = 0
temp0_total_yellow = 0
temp1_total_yellow = 0
# 逐一讀取並合併所有Parquet檔案
print('fhvfv==初始筆數=剩餘筆數=移除佔比')
for file in datasite_fhvhv:
    # 使用pandas的read_parquet函數讀取Parquet檔案並將其轉換為DataFrame
    df = pd.read_parquet(file)
    temp_0 = df.shape[0]
    temp0_total_fhvfv += temp_0
    # 留下需要的欄位
    df =  df[['hvfhs_license_num','pickup_datetime','dropoff_datetime','PULocationID','DOLocationID','trip_miles','trip_time','shared_match_flag']]
    # 移除非當月的data
    df =  df[(df['pickup_datetime'].dt.year == data_year) & (df['pickup_datetime'].dt.month == data_month)]
    temp_1 = df.shape[0]
    temp1_total_fhvfv += temp_1
    # 查看剩餘資料量
    if temp_0 > temp_1 :
        print(data_year,'=',data_month,'=',temp_0,'=',temp_1,round(100-(temp_1/temp_0*100),5),"%")
    else:
        print(data_year,'=',data_month,'=',temp_0,'=',temp_1,'=0%')
    # 年月處理
    if data_month == 12:
        data_year += 1
        data_month = 1
    else :
        data_month += 1
    # 將數據合併到已有的DataFrame中
    df_fhvhv = pd.concat([df_fhvhv, df], ignore_index=True)
    
    

data_year = set_data_year
data_month = set_data_month
print('yellow==初始筆數=剩餘筆數=移除佔比')
for file in datasite_yellow:
    # 使用pandas的read_parquet函數讀取Parquet檔案並將其轉換為DataFrame
    df = pd.read_parquet(file)
    temp_0 = df.shape[0]
    temp0_total_yellow += temp_0
    # 留下需要的欄位
    df =  df[['tpep_pickup_datetime','tpep_dropoff_datetime','trip_distance','PULocationID','DOLocationID']]
    # 移除非當月的data
    df =  df[(df['tpep_pickup_datetime'].dt.year == data_year) & (df['tpep_pickup_datetime'].dt.month == data_month)]
    temp_1 = df.shape[0]
    temp1_total_yellow += temp_1
    # 查看剩餘資料量
    if temp_0 > temp_1 :
        print(data_year,'=',data_month,'=',temp_0,'=',temp_1,'=',round(100-(temp_1/temp_0*100),5),"%")
    else:
        print(data_year,'=',data_month,'=',temp_0,'=',temp_1,'=0%')        
    # 年月處理
    if data_month == 12:
        data_year += 1
        data_month = 1
    else :
        data_month += 1
    # 將數據合併到已有的DataFrame中
    df_yellow = pd.concat([df_yellow, df], ignore_index=True)
    
    
print(f'fhvfv={start_year}=初始總筆數=剩餘總筆數=移除佔比')
if temp0_total_fhvfv > temp1_total_fhvfv :
    print('\==',temp0_total_fhvfv,'=',temp1_total_fhvfv,'=',round(100-(temp1_total_fhvfv/temp0_total_fhvfv*100),5),"%")
else:
    print('\==',temp0_total_fhvfv,'=',temp1_total_fhvfv,'=0%')

print(f'yellow={start_year}=初始總筆數=剩餘總筆數=移除佔比')
if temp0_total_yellow > temp1_total_yellow :
    print('\==',temp0_total_yellow,'=',temp1_total_yellow,'=',round(100-(temp1_total_yellow/temp0_total_yellow*100),5),"%")
else:
    print('\==',temp0_total_yellow,'=',temp1_total_yellow,'=0%')

# 記錄結束時間
end_time = time.time()
# 計算程式執行花費的時間
execution_time = end_time - start_time
print("\n程式執行花費的時間：", round(execution_time,2), "秒")


#清理暫存
del data_month,data_year,df,temp_0,temp_1,temp0_total_yellow,temp1_total_yellow
print("done時間:", datetime.datetime.now())

fhvfv==初始筆數=剩餘筆數=移除佔比
2020 = 1 = 20569368 = 20569368 =0%
2020 = 2 = 21725100 = 21725100 =0%
2020 = 3 = 13392928 = 13392928 =0%
2020 = 4 = 4312909 = 4312909 =0%
2020 = 5 = 6089999 = 6089999 =0%
2020 = 6 = 7555193 = 7555193 =0%
2020 = 7 = 9958454 = 9958454 =0%
2020 = 8 = 11096852 = 11096852 =0%
2020 = 9 = 12106669 = 12106669 =0%
2020 = 10 = 13268411 = 13268411 =0%
2020 = 11 = 11596865 = 11596865 =0%
2020 = 12 = 11637123 = 11637123 =0%
yellow==初始筆數=剩餘筆數=移除佔比
2020 = 1 = 6405008 = 6404796 = 0.00331 %
2020 = 2 = 6299367 = 6299064 = 0.00481 %
2020 = 3 = 3007687 = 3007261 = 0.01416 %
2020 = 4 = 238073 = 237941 = 0.05545 %
2020 = 5 = 348415 = 348392 = 0.0066 %
2020 = 6 = 549797 = 549788 = 0.00164 %
2020 = 7 = 800412 = 800402 = 0.00125 %
2020 = 8 = 1007286 = 1007270 = 0.00159 %
2020 = 9 = 1341017 = 1340883 = 0.00999 %
2020 = 10 = 1681132 = 1681100 = 0.0019 %
2020 = 11 = 1509000 = 1508888 = 0.00742 %
2020 = 12 = 1461898 = 1461863 = 0.00239 %
fhvfv=2020=初始總筆數=剩餘總筆數=移除佔比
\== 143309871 = 143309871 =

In [4]:
df_fhvhv

,hvfhs_license_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,shared_match_flag
0,HV0003,2020-01-01 00:45:34,2020-01-01 01:02:20,148,90,1.930,1006,N
1,HV0003,2020-01-01 00:47:50,2020-01-01 00:53:23,114,79,0.810,333,N
2,HV0003,2020-01-01 00:04:37,2020-01-01 00:21:49,4,125,2.530,1032,N
3,HV0003,2020-01-01 00:26:36,2020-01-01 00:33:00,231,113,1.110,384,N
4,HV0003,2020-01-01 00:37:49,2020-01-01 00:46:59,114,144,1.100,550,N
...,...,...,...,...,...,...,...,...
143309866,HV0003,2020-12-31 23:28:52,2020-12-31 23:35:07,42,247,1.170,375,N
143309867,HV0003,2020-12-31 23:39:46,2020-12-31 23:50:35,247,60,2.130,649,N
143309868,HV0005,2020-12-31 23:02:01,2020-12-31 23:09:58,76,77,1.693,477,N
143309869,HV0005,2020-12-31 23:12:45,2020-12-31 23:26:20,77,91,3.262,815,N


In [5]:
df_yellow

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID
0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.20,238,239
1,2020-01-01 00:35:39,2020-01-01 00:43:04,1.20,239,238
2,2020-01-01 00:47:41,2020-01-01 00:53:52,0.60,238,238
3,2020-01-01 00:55:23,2020-01-01 01:00:14,0.80,238,151
4,2020-01-01 00:01:58,2020-01-01 00:04:16,0.00,193,193
...,...,...,...,...,...
24647643,2020-12-31 23:44:35,2021-01-01 00:01:22,9.22,143,20
24647644,2020-12-31 23:41:36,2020-12-31 23:50:32,4.79,4,262
24647645,2020-12-31 23:01:17,2020-12-31 23:40:37,28.00,210,78
24647646,2020-12-31 23:31:29,2020-12-31 23:44:22,7.08,148,129


### 特徵整理,統一命名
HVFHV:'hvfhs_license_num','pickup_datetime','dropoff_datetime','PULocationID','DOLocationID','trip_miles','trip_time'
Yellow:'tpep_pickup_datetime','tpep_dropoff_datetime','trip_distance','PULocationID','DOLocationID'


    newname (HVFHV/Yellow/Green)
    1.TLC 許可證號 : Name(hvfhs_license_num/--/--)
        *改為名稱標記 HV0003-uber/HV0005-lyft/yellow
    2.上車時間 : BoardingTime (pickup_datetime/tpep_pickup_datetime/lpep_pickup_datetime)
    3.下車時間 : DisembarkingTime (dropoff_datetime/tpep_dropoff_datetime/lpep_dropoff_datetime)
    4.上車地點ID : PULocationID (PULocationID/PULocationID/PULocationID)
    5.下車地點ID : DOLocationID (DOLocationID/DOLocationID/DOLocationID)
    6.行駛里程 : TripMiles (trip_miles/trip_distance/trip_distance)
    7.行駛時間(秒) : TripTime (trip_time/--/--)
        *驗證是否正確?
        *下車時間-上車時間
    8.共乘 : shared_match_flag (shared_match_flag/--/--)
    9.節日 : is_holiday(--/--/--)
    10.星期 : weekday(--/--/--)

In [6]:
# 記錄開始時間
start_time = time.time()

df_TLC = pd.DataFrame()
#名稱統一
rename_dict_fhvhv = {'hvfhs_license_num':'Name','pickup_datetime': 'BoardingTime', 'dropoff_datetime': 'DisembarkingTime', 'PULocationID': 'PULocationID','DOLocationID':'DOLocationID', 'trip_miles': 'TripMiles', 'trip_time': 'TripTime', 'base_passenger_fare': 'BaseFare','shared_match_flag':'shared_match_flag'}
df_fhvhv.rename(columns=rename_dict_fhvhv, inplace=True)
rename_dict_yellow = {'tpep_pickup_datetime': 'BoardingTime', 'tpep_dropoff_datetime': 'DisembarkingTime', 'PULocationID': 'PULocationID','DOLocationID':'DOLocationID', 'trip_distance': 'TripMiles', 'fare_amount': 'BaseFare'}
df_yellow.rename(columns=rename_dict_yellow, inplace=True)

#fhvhv_Name
df_fhvhv.loc[df_fhvhv['Name'] == 'HV0003', 'Name'] = 'uber'
df_fhvhv.loc[df_fhvhv['Name'] == 'HV0005', 'Name'] = 'lyft'

#yellow 新增特徵 TripTime,Name
df_yellow['TripTime'] = (df_yellow['DisembarkingTime'] - df_yellow['BoardingTime']).dt.total_seconds()
df_yellow['Name'] = 'yellow'
df_yellow['shared_match_flag'] = 'N'
#yellow合併到fhvhv
df_yellow = df_yellow.reindex(columns = df_fhvhv.columns)
df_TLC = pd.concat([df_fhvhv, df_yellow], ignore_index=True)


# 記錄結束時間
end_time = time.time()
# 計算程式執行花費的時間
execution_time = end_time - start_time
print("程式執行花費的時間：", round(execution_time,2), "秒")

#清理暫存
del df_yellow ,df_fhvhv
print("done時間:", datetime.datetime.now())

程式執行花費的時間： 18.1 秒
done時間: 2023-08-16 13:52:18.340999


### 紀錄初始資料筆數

In [7]:
#紀錄初始資料筆數
init_df_TLC_shape = df_TLC.shape
init_df_uber_shape = df_TLC[df_TLC['Name']=='uber'].shape
init_df_lyft_shape = df_TLC[df_TLC['Name']=='lyft'].shape
init_df_yellow_shape = df_TLC[df_TLC['Name']=='yellow'].shape
print('Name=init_data_row=init_data_column')
print(f'TLC={init_df_TLC_shape[0]}={init_df_TLC_shape[1]}')
print(f'uber={init_df_uber_shape[0]}={init_df_uber_shape[1]}')
print(f'lyft={init_df_lyft_shape[0]}={init_df_lyft_shape[1]}')
print(f'yellow={init_df_yellow_shape[0]}={init_df_yellow_shape[1]}')

print("\ndone時間:", datetime.datetime.now())

Name=init_data_row=init_data_column
TLC=167957519=8
uber=103184679=8
lyft=37252636=8
yellow=24647648=8

done時間: 2023-08-16 13:56:40.048531


### 缺失值確認

In [8]:
print(df_TLC.isnull().sum())

Name                 0
BoardingTime         0
DisembarkingTime     0
PULocationID         0
DOLocationID         0
TripMiles            0
TripTime             0
shared_match_flag    0
dtype: int64


### 移除非目標車商

In [9]:
#查看目前有的車商
print(df_TLC['Name'].unique())

print("\ndone時間:", datetime.datetime.now())

['uber' 'lyft' 'HV0004' 'yellow']

done時間: 2023-08-16 13:58:00.772658


In [10]:
#筆數計算-1
print('HV0002=筆數=占比')
print('\=', df_TLC[(df_TLC['Name'] == 'HV0002')].shape[0],'=',round(df_TLC[(df_TLC['Name'] == 'HV0002')].shape[0] / init_df_TLC_shape[0]*100,4),'%')

print("\ndone時間:", datetime.datetime.now())

HV0002=筆數=占比
\= 0 = 0.0 %

done時間: 2023-08-16 13:58:15.806432


In [12]:
#筆數計算-2
print('HV0004=筆數=占比')
print('\=', df_TLC[(df_TLC['Name'] == 'HV0004')].shape[0],'=',round(df_TLC[(df_TLC['Name'] == 'HV0004')].shape[0] / init_df_TLC_shape[0]*100,4),'%')

print("\ndone時間:", datetime.datetime.now())

HV0004=筆數=占比
\= 2872556 = 1.7103 %

done時間: 2023-08-16 13:58:55.574328


In [13]:
#移除
df_TLC = df_TLC.drop(df_TLC[df_TLC['Name'] == 'HV0002'].index)
df_TLC = df_TLC.drop(df_TLC[df_TLC['Name'] == 'HV0004'].index)
print(df_TLC['Name'].unique())

print("\ndone時間:",datetime.datetime.now())

['uber' 'lyft' 'yellow']

done時間: 2023-08-16 14:00:21.183552


### 移除上車地點 PULocationID 未知 (264,265)

In [14]:
#筆數計算
print('264筆數=265筆數=總占比') 
print(df_TLC[(df_TLC['PULocationID'] == 264)].shape[0],'=', df_TLC[(df_TLC['PULocationID'] == 265)].shape[0],'=', 
       round(df_TLC[(df_TLC['PULocationID'] == 264) | (df_TLC['PULocationID'] == 265)].shape[0] / init_df_TLC_shape[0]*100,4),'%'  )

print("\ndone時間:", datetime.datetime.now())

264筆數=265筆數=總占比
166687 = 67091 = 0.1392 %

done時間: 2023-08-16 14:00:25.405934


In [15]:
df_TLC = df_TLC[~df_TLC['PULocationID'].isin([264, 265])]

#check-必須為0
print('上車未知剩餘筆數 = ',df_TLC[(df_TLC['PULocationID'] == 264) | (df_TLC['PULocationID'] == 265)].shape[0])
print("\ndone時間:", datetime.datetime.now())

上車未知剩餘筆數 =  0

done時間: 2023-08-16 14:00:35.984144


### 移除不符合的共乘資料
    UBER 2020/3/17-2022/6/20
    LYFT 2020/3/17-2021/8/18
         2023/5/12-

In [16]:
#uber,lyft
# 定義起始日期和結束日期
uber_start_date = pd.to_datetime('2020-03-17')
uber_end_date = pd.to_datetime('2022-06-20')
lyft_start_date = pd.to_datetime('2020-03-17')
lyft_end_date = pd.to_datetime('2021-08-18')
lyft_start_date1 = pd.to_datetime('2023-05-12')

#筆數計算
temp_uber = ((df_TLC['Name'] == 'uber') & (df_TLC['shared_match_flag'] == 'Y') & (df_TLC['BoardingTime'].between(uber_start_date, uber_end_date))).sum()

temp_lyft = ((df_TLC['Name'] == 'lyft') & (df_TLC['shared_match_flag'] == 'Y') & ((df_TLC['BoardingTime'].between(lyft_start_date, lyft_end_date))|(df_TLC['BoardingTime'] >= lyft_start_date1 ))).sum()
print('/=uber=lyft')
print('data=',temp_uber,'=',temp_lyft)
print('占比=',round(temp_uber / init_df_uber_shape[0]*100,4),'%','=',round(temp_uber / init_df_uber_shape[0]*100,4),'%' )

print("\ndone時間:", datetime.datetime.now())

#清理暫存
del temp_uber ,temp_lyft

/=uber=lyft
data= 1056 = 3704
占比= 0.001 % = 0.001 %

done時間: 2023-08-16 14:01:04.522144


In [17]:
# 過濾條件：shared_match_flag為Y且BoardingTime介於指定日期之間
df_TLC = df_TLC[~ ((df_TLC['Name'] == 'uber') & (df_TLC['shared_match_flag'] == 'Y') & (df_TLC['BoardingTime'].between(uber_start_date, uber_end_date)))]
df_TLC = df_TLC[~ ((df_TLC['Name'] == 'lyft') & (df_TLC['shared_match_flag'] == 'Y') & ((df_TLC['BoardingTime'].between(lyft_start_date, lyft_end_date))|(df_TLC['BoardingTime'] >= lyft_start_date1 )))]


#check-必須為0
print('uber不符合的共乘資料剩餘筆數 = ',df_TLC[(df_TLC['Name'] == 'uber') & (df_TLC['shared_match_flag'] == 'Y') & (df_TLC['BoardingTime'].between(uber_start_date, uber_end_date))].shape[0]  )
print('lyft不符合的共乘資料剩餘筆數 = ',df_TLC[(df_TLC['Name'] == 'lyft') & (df_TLC['shared_match_flag'] == 'Y') & ((df_TLC['BoardingTime'].between(lyft_start_date, lyft_end_date))|(df_TLC['BoardingTime'] >= lyft_start_date1 ))].shape[0]  )
print("\ndone時間:", datetime.datetime.now())

#清理暫存
del uber_start_date ,uber_end_date,lyft_start_date,lyft_end_date,lyft_start_date1

uber不符合的共乘資料剩餘筆數 =  0
lyft不符合的共乘資料剩餘筆數 =  0

done時間: 2023-08-16 14:02:16.129807


###　不處理  trip_time != DisembarkingTime - BoardingTime
### 移除 TripTime <= 0

In [18]:
#記錄開始時間
start_time = time.time()

#印出資訊
temp_uber = ((df_TLC['Name'] == 'uber') & (df_TLC['TripTime'] <= 0)).sum()
temp_lyft = ((df_TLC['Name'] == 'lyft') & (df_TLC['TripTime'] <= 0)).sum()
temp_yellow = ((df_TLC['Name'] == 'yellow') & (df_TLC['TripTime'] <= 0)).sum()

print('\=uber=lyft=yellow')
print('data=',temp_uber,'=',temp_lyft,'=',temp_yellow)
print('占比=',round(temp_uber / init_df_uber_shape[0]*100,5),'%='
              ,round(temp_lyft / init_df_lyft_shape[0]*100,5),'%='
              ,round(temp_yellow / init_df_yellow_shape[0]*100,5),'%' )

#記錄結束時間
end_time = time.time()
#計算程式執行花費的時間
execution_time = end_time - start_time
print("\n程式執行花費的時間：", round(execution_time,2), "秒")
print("done時間:", datetime.datetime.now())

\=uber=lyft=yellow
data= 41 = 48 = 15178
占比= 4e-05 %= 0.00013 %= 0.06158 %

程式執行花費的時間： 18.55 秒
done時間: 2023-08-16 14:02:42.920228


In [19]:
#移除 (df_TLC['TripTime'] <= 0)
df_TLC = df_TLC.drop(df_TLC[df_TLC['TripTime'] <= 0].index)
#check-必須為0
print('TripTime<=0剩餘筆數:',(df_TLC['TripTime'] <= 0).sum())
print("\ndone時間:", datetime.datetime.now())

TripTime<=0剩餘筆數: 0

done時間: 2023-08-16 14:03:22.959144


### 移除TripMiles <= 0

In [20]:
#記錄開始時間
start_time = time.time()

#印出資訊
temp_uber = ((df_TLC['Name'] == 'uber') & (df_TLC['TripMiles'] <= 0)).sum()
temp_lyft = ((df_TLC['Name'] == 'lyft') & (df_TLC['TripMiles'] <= 0)).sum()
temp_yellow = ((df_TLC['Name'] == 'yellow') & (df_TLC['TripMiles'] <= 0)).sum()

print('\=uber=lyft=yellow')
print('data=',temp_uber,'=',temp_lyft,'=',temp_yellow)
print('占比=',round(temp_uber / init_df_uber_shape[0]*100,5),'%='
              ,round(temp_lyft / init_df_lyft_shape[0]*100,5),'%='
              ,round(temp_yellow / init_df_yellow_shape[0]*100,5),'%' )


#記錄結束時間
end_time = time.time()
#計算程式執行花費的時間
execution_time = end_time - start_time
print("\n程式執行花費的時間：", round(execution_time,2), "秒")

del temp_uber,temp_lyft,temp_yellow
print("done時間:",datetime.datetime.now())

\=uber=lyft=yellow
data= 12804 = 126165 = 295626
占比= 0.01241 %= 0.33867 %= 1.19941 %

程式執行花費的時間： 19.24 秒
done時間: 2023-08-16 14:03:46.559263


In [21]:
#移除 (df_TLC['TripMiles'] <= 0)
df_TLC = df_TLC.drop(df_TLC[df_TLC['TripMiles'] <= 0].index)
#check-必須為0
print('TripMiles<=0剩餘筆數:',(df_TLC['TripMiles'] <= 0).sum())
print("\ndone時間:", datetime.datetime.now())

TripMiles<=0剩餘筆數: 0

done時間: 2023-08-16 14:04:17.561318


### 新增節日相關資訊

In [22]:
# from holidays.calendars import gregorian
class AllHolidays(US):
    def _populate(self, year):
        super()._populate(year)
        self.pop_named("Lincoln's Birthday")
        self.pop_named("Susan B. Anthony Day")
        
        self._add_holiday("Valentine's Day", 2, 14)
        self._add_holiday("St.Patrick's Day", 3, 17)
        self._add_holiday("Halloween", 10, 31)
        
#        self._add_holiday("Thanksgiving weekend", 
#                          self._get_nth_weekday_of_month(month=11, n=4, weekday=4))
#        self._add_holiday("Thanksgiving weekend", 
#                          self._get_nth_weekday_of_month(month=11, n=4, weekday=5))
#        self._add_holiday("Thanksgiving weekend", 
#                          self._get_nth_weekday_of_month(month=11, n=4, weekday=6))
        
        self._add_holiday("Christmas Eve", 12, 24)
        self._add_holiday("Christmas season", 12, 26)
        self._add_holiday("Christmas season", 12, 27)
        self._add_holiday("Christmas season", 12, 28)
        self._add_holiday("Christmas season", 12, 29)
        self._add_holiday("Christmas season", 12, 30)
        
        self._add_holiday("New Year's Eve", 12, 31)

# 實例化AllHolidays
ny_holidays_all = AllHolidays(subdiv='NY', years=[2019, 2020, 2021, 2022, 2023])

# 新增2019~2023年的復活節、復活節星期一
ny_holidays_all[datetime.date(2019, 4, 21)] = "Easter"
ny_holidays_all[datetime.date(2019, 4, 22)] = "Easter Monday"
ny_holidays_all[datetime.date(2020, 4, 12)] = "Easter"
ny_holidays_all[datetime.date(2020, 4, 13)] = "Easter Monday"
ny_holidays_all[datetime.date(2021, 4, 4)] = "Easter"
ny_holidays_all[datetime.date(2021, 4, 5)] = "Easter Monday"
ny_holidays_all[datetime.date(2022, 4, 17)] = "Easter"
ny_holidays_all[datetime.date(2022, 4, 18)] = "Easter Monday"
ny_holidays_all[datetime.date(2023, 4, 9)] = "Easter"
ny_holidays_all[datetime.date(2023, 4, 10)] = "Easter Monday"

# 新增2019~2023感恩節
ny_holidays_all[datetime.date(2019,11, 28)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2019,11, 29)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2019,11, 30)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2020,11, 26)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2020,11, 27)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2020,11, 28)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2021,11, 25)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2021,11, 26)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2021,11, 27)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2022,11, 24)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2022,11, 25)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2022,11, 26)] =  "Thanksgiving weekend"
ny_holidays_all[datetime.date(2023,11, 23)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2023,11, 24)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2023,11, 25)] = "Thanksgiving weekend"

# 查看
for i in sorted(ny_holidays_all.items()):
    print(i)

del i

(datetime.date(2019, 1, 1), "New Year's Day")
(datetime.date(2019, 1, 21), 'Martin Luther King Jr. Day')
(datetime.date(2019, 2, 14), "Valentine's Day")
(datetime.date(2019, 2, 18), "Washington's Birthday")
(datetime.date(2019, 3, 17), "St.Patrick's Day")
(datetime.date(2019, 4, 21), 'Easter')
(datetime.date(2019, 4, 22), 'Easter Monday')
(datetime.date(2019, 5, 27), 'Memorial Day')
(datetime.date(2019, 7, 4), 'Independence Day')
(datetime.date(2019, 9, 2), 'Labor Day')
(datetime.date(2019, 10, 14), 'Columbus Day')
(datetime.date(2019, 10, 31), 'Halloween')
(datetime.date(2019, 11, 5), 'Election Day')
(datetime.date(2019, 11, 11), 'Veterans Day')
(datetime.date(2019, 11, 28), 'Thanksgiving; Thanksgiving weekend')
(datetime.date(2019, 11, 29), 'Thanksgiving weekend')
(datetime.date(2019, 11, 30), 'Thanksgiving weekend')
(datetime.date(2019, 12, 24), 'Christmas Eve')
(datetime.date(2019, 12, 25), 'Christmas Day')
(datetime.date(2019, 12, 26), 'Christmas season')
(datetime.date(2019, 12, 

In [23]:
# 增加所有節日標籤
# all_data為Uber, Lyft, Yellow合併後的DataFrame，再請自行修改名稱
df_TLC['is_holiday'] = np.where(df_TLC['BoardingTime'].dt.date.isin(ny_holidays_all), True, False)
print(df_TLC['is_holiday'].value_counts())

print('-------------')
print('Name=True=False')
print('uber=',((df_TLC['Name'] == 'uber') & (df_TLC['is_holiday'] == True)).sum(),'=',((df_TLC['Name'] == 'uber') & (df_TLC['is_holiday'] == False)).sum())
print('lyft=',((df_TLC['Name'] == 'lyft') & (df_TLC['is_holiday'] == True)).sum(),'=',((df_TLC['Name'] == 'lyft') & (df_TLC['is_holiday'] == False)).sum())
print('yellow=',((df_TLC['Name'] == 'yellow') & (df_TLC['is_holiday'] == True)).sum(),'=',((df_TLC['Name'] == 'yellow') & (df_TLC['is_holiday'] == False)).sum())
print('Total=',(df_TLC['is_holiday'] == True).sum(),'=',(df_TLC['is_holiday'] == False).sum())

print("\ndone時間:", datetime.datetime.now())



is_holiday
False    153045955
True      11350608
Name: count, dtype: int64
-------------
Name=True=False
uber= 7272951 = 95892217
lyft= 2646055 = 34473067
yellow= 1431602 = 22680671
Total= 11350608 = 153045955

done時間: 2023-08-16 14:05:52.643687


### 新增星期特徵
    #星期幾 (用上車時間 BoardingTime)
    #(一,二,三,四,五,六,日) = (0,1,2,3,4,5,6)

In [24]:
#記錄開始時間
start_time = time.time()

df_TLC['weekday'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.weekday
df_TLC['date'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.date
df_TLC['year'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.year
df_TLC['month'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.month
df_TLC['day'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.day
df_TLC['hour'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.hour

#記錄結束時間
end_time = time.time()
#計算程式執行花費的時間
execution_time = end_time - start_time
print("程式執行花費的時間：", round(execution_time,2), "秒")
print("done時間:", datetime.datetime.now())

程式執行花費的時間： 72.26 秒
done時間: 2023-08-16 14:07:26.937793


### 時間序列
    'Name','date','year','month','day','hour','weekday','PULocationID','DOLocationID',
    'TripMiles','TripTime','shared_match_flag'

In [25]:
#記錄開始時間
start_time = time.time()

dfTime_TLC = df_TLC.groupby(['Name','year','month','day',
                     'hour','weekday','PULocationID','is_holiday'
                  ]).size().reset_index(name="次數")

#記錄結束時間
end_time = time.time()
#計算程式執行花費的時間
execution_time = end_time - start_time
print("程式執行花費的時間：", round(execution_time,2), "秒")
print("done時間:", datetime.datetime.now())

程式執行花費的時間： 40.75 秒
done時間: 2023-08-16 14:09:09.253521


In [26]:
print('row=column')
print(dfTime_TLC.shape[0],'=',dfTime_TLC.shape[1])
dfTime_TLC

row=column
5065678 = 9


,Name,year,month,day,hour,weekday,PULocationID,is_holiday,次數
0,lyft,2020,1,1,0,2,3,True,11
1,lyft,2020,1,1,0,2,4,True,35
2,lyft,2020,1,1,0,2,5,True,1
3,lyft,2020,1,1,0,2,6,True,2
4,lyft,2020,1,1,0,2,7,True,138
...,...,...,...,...,...,...,...,...,...
5065673,yellow,2020,12,31,23,3,256,True,2
5065674,yellow,2020,12,31,23,3,260,True,2
5065675,yellow,2020,12,31,23,3,261,True,3
5065676,yellow,2020,12,31,23,3,262,True,9


In [27]:
print('row=column')
print(df_TLC.shape[0],'=',df_TLC.shape[1])
df_TLC

row=column
164396563 = 15


,Name,BoardingTime,DisembarkingTime,PULocationID,DOLocationID,TripMiles,TripTime,shared_match_flag,is_holiday,weekday,date,year,month,day,hour
0,uber,2020-01-01 00:45:34,2020-01-01 01:02:20,148,90,1.93,1006.0,N,True,2,2020-01-01,2020,1,1,0
1,uber,2020-01-01 00:47:50,2020-01-01 00:53:23,114,79,0.81,333.0,N,True,2,2020-01-01,2020,1,1,0
2,uber,2020-01-01 00:04:37,2020-01-01 00:21:49,4,125,2.53,1032.0,N,True,2,2020-01-01,2020,1,1,0
3,uber,2020-01-01 00:26:36,2020-01-01 00:33:00,231,113,1.11,384.0,N,True,2,2020-01-01,2020,1,1,0
4,uber,2020-01-01 00:37:49,2020-01-01 00:46:59,114,144,1.10,550.0,N,True,2,2020-01-01,2020,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167957514,yellow,2020-12-31 23:44:35,2021-01-01 00:01:22,143,20,9.22,1007.0,N,True,3,2020-12-31,2020,12,31,23
167957515,yellow,2020-12-31 23:41:36,2020-12-31 23:50:32,4,262,4.79,536.0,N,True,3,2020-12-31,2020,12,31,23
167957516,yellow,2020-12-31 23:01:17,2020-12-31 23:40:37,210,78,28.00,2360.0,N,True,3,2020-12-31,2020,12,31,23
167957517,yellow,2020-12-31 23:31:29,2020-12-31 23:44:22,148,129,7.08,773.0,N,True,3,2020-12-31,2020,12,31,23


### 輸出為一份新檔案

In [28]:
custom_output_path = custom_output_path_dfTime

dfTime_TLC.to_parquet(f'{custom_output_path}/dfTime_TLC-{set_data_year}.parquet')
print("done時間:", datetime.datetime.now())

done時間: 2023-08-16 14:13:01.855294


In [29]:
custom_output_path = custom_output_path_dfTLC

df_TLC.to_parquet(f'{custom_output_path}/TLC-{set_data_year}.parquet')
print("done時間:", datetime.datetime.now())

done時間: 2023-08-16 14:14:39.877283
